In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
import flor
flor.__file__

'/home/ubuntu/flor/flor/__init__.py'

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [4]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    if flor.SkipBlock.step_into('training_loop'):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % args.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
                if args.dry_run:
                    break
    flor.SkipBlock.end(model, optimizer)


In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# I show you a piece of code, and you tell me, what do you want to know about it?

In [ ]:
flor.flags.NAME = 'cornell-demo'
class Args:
    batch_size = 64
    test_batch_size = 1000
    epochs = 14
    lr = 1.0
    gamma = 0.7
    no_cuda = False
    dry_run = False
    log_interval = 10
    save_model = False
    
args = Args
use_cuda = (not args.no_cuda) and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in flor.it(range(1, args.epochs + 1)):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.325038
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.534244
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.965782
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.434723
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.369582
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.263104
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.244222
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.313011
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.256713
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.389653
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.222857
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.250901
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.104127
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.238097
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.194373
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.346091
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.054728
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.123539
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.270559
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.077767
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.148230
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.041499
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.144036
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.043660
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.077967
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.116173
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.083221
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.105464
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.083224
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.071551
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.011271
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.043133
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.056182
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.068612
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.094379
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.124000
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.045354
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.117006
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.037054
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.071250


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.015569
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.059980
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.026247
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.019301
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.019736
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.005470
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.017186
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.024458
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.022824
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.029101
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.015313
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.064916
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.014506
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.035623
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.002650
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.014069
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.015424
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.086426
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.029387
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.047631


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.013335
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.050604
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.019744
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.017763
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.003322
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.070333
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.030034
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.038642
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.033338
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.056382
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.009784
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.003999
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.011437
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.017043
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.004327
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.026600
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.056329
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.061920
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.015758
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.084062


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.005123
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.055528
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.002746
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.050807

Test set: Average loss: 0.0278, Accuracy: 9917/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.026843
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.042378
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.020900
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.011945
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.020000
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.010683
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.056054
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.006708
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.032303
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.081955
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.046194
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.063279
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.036688
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.015614
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.012685
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.011440
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.031741
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.067905
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.202904
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.013698
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.014028
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.003415
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.201970
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.019389
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.003245
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.005215
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.010436
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.011451
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.002850
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.018661
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.016423
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.016862
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.022411
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.003634
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.161178
